In [1]:
import pandas as pd

# SCHEMI, ANNOTAZIONI, GS

In [6]:
url = "https://raw.githubusercontent.com/GConte47/tirocinio/master/df%20omap/files/SCHEMI.csv"

SCHEMI = pd.read_csv(url, delimiter=",")  # Cambia il delimitatore se non è ";"

url = "https://raw.githubusercontent.com/GConte47/tirocinio/refs/heads/master/df%20omap/gold_standard.csv"

GS = pd.read_csv(url, delimiter= ",")


url = "https://raw.githubusercontent.com/GConte47/tirocinio/refs/heads/master/df%20omap/files/medcat/Medcat_SNOMED_Df_CS_08_ANC.csv"
EsempioAnnotazione = pd.read_csv(url, delimiter=",")

In [2]:
#versione locale
SCHEMI = pd.read_csv('files/SCHEMI.csv', delimiter=",")
GS = pd.read_csv('files/gold_standard.csv', delimiter=",")
EsempioAnnotazione = pd.read_csv('files/medcat/Medcat_SNOMED_Df_CS_08_ANC.csv', delimiter=",")

In [3]:
# controllo coerenza

SCHEMI.columns, GS.columns

(Index(['SOURCE', 'AttributoNomeCompleto', 'DescrizioneCompleta',
        'DescrizioneTabella', 'DescrizioneAttributo', 'Tabella', 'Attributo'],
       dtype='object'),
 Index(['TipoDiAnnotazione', 'Ontologies', 'Source', 'AttributoNomeCompleto',
        'entity_id', 'pretty_name', 'cui'],
       dtype='object'))

In [4]:
# occorre   usare gli stessi attributi nomi per lo stesso concetto
GS.rename(columns={"Source": "SOURCE"}, inplace=True)


In [5]:
# in GS 'SOURCE', 'AttributoNomeCompleto' deve essere una foreign key riferita a SCHEMI
len(GS), len(pd.merge(GS,SCHEMI))

(323, 188)

In [6]:
diff = GS[['SOURCE', 'AttributoNomeCompleto']].merge(
    SCHEMI[['SOURCE', 'AttributoNomeCompleto']],
    on=['SOURCE', 'AttributoNomeCompleto'],
    how='left',
    indicator=True
)

# Seleziona solo le coppie presenti in GS ma non in SCHEMI
not_in_schemi = diff[diff['_merge'] == 'left_only'].drop(columns=['_merge'])

not_in_schemi

,SOURCE,AttributoNomeCompleto
0,OMOP,location-location
1,OMOP,location-location
2,OMOP,location-location
3,OMOP,location-location
4,OMOP,location-location
...,...,...
253,OMOP,person-datetime_of_birth
254,OMOP,person-datetime_of_death
255,OMOP,person-datetime_of_death
256,OMOP,person-datetime_of_death


In [7]:
diff = GS[['SOURCE', 'AttributoNomeCompleto']].merge(
    SCHEMI[['SOURCE', 'AttributoNomeCompleto']],
    on=[ 'AttributoNomeCompleto'],
    how='left',
    indicator=True
)

# Seleziona solo le coppie presenti in GS ma non in SCHEMI
diff[diff['_merge'] == 'left_only'].drop(columns=['_merge'])

,SOURCE_x,AttributoNomeCompleto,SOURCE_y
0,OMOP,location-location,NaN
1,OMOP,location-location,NaN
2,OMOP,location-location,NaN
3,OMOP,location-location,NaN
4,OMOP,location-location,NaN
...,...,...,...
253,OMOP,person-datetime_of_birth,NaN
254,OMOP,person-datetime_of_death,NaN
255,OMOP,person-datetime_of_death,NaN
256,OMOP,person-datetime_of_death,NaN


In [8]:
# vediamo perchè mancano
SCHEMI[SCHEMI['AttributoNomeCompleto'].str.contains('location', na=False, case=False)]

,SOURCE,AttributoNomeCompleto,DescrizioneCompleta,DescrizioneTabella,DescrizioneAttributo,Tabella,Attributo
9,OMOP,person-location_id,the person domain contains records that unique...,the person domain contains records that unique...,a foreign key to the place of residency for th...,person,location_id
127,OMOP,location-location_id,the location table represents a generic way to...,the location table represents a generic way to...,a unique identifier for each geographic location.,location,location_id
128,OMOP,location-address_1,the location table represents a generic way to...,the location table represents a generic way to...,"the address field 1, typically used for the st...",location,address_1
129,OMOP,location-address_2,the location table represents a generic way to...,the location table represents a generic way to...,"the address field 2, typically used for additi...",location,address_2
130,OMOP,location-city,the location table represents a generic way to...,the location table represents a generic way to...,the city field as it appears in the source data.,location,city
131,OMOP,location-state,the location table represents a generic way to...,the location table represents a generic way to...,the state field as it appears in the source data.,location,state
132,OMOP,location-zip,the location table represents a generic way to...,the location table represents a generic way to...,the zip or postal code.,location,zip
133,OMOP,location-county,the location table represents a generic way to...,the location table represents a generic way to...,the county.,location,county
134,OMOP,location-location_source_value,the location table represents a generic way to...,the location table represents a generic way to...,the verbatim information that is used to uniqu...,location,location_source_value
240,OMOP,care_site-location_id,the care_site table contains a list of uniquel...,the care_site table contains a list of uniquel...,a foreign key to the geographic location in th...,care_site,location_id


In [21]:
# location-location 	 dovrebbe corrispondere a location-location_id
# RISPOSTA= location-location in realtà dovrebbe corrispondere solo a location dato che i codici sono corrispondenti alla tabella principale.

# location-address 	possiamo dire che corrisponde a location-address_1
#RISPOSTA= no, corrisponde a location address_1 e address_2. è un errore mio che li ho omessi per sbaglio dal gold standard (PROBLEMA RISOLTO)

# location-location_source-value 	 c'è anche in SCHEMI location-location_source_value, forse c'è qualche carattere strano nella stringa
# vedi con chatGPT come controllare
# anche qui un mio errore di battitura. l'errore è che anzichè mettere '_' ho messo '-' (PROBLEMA RISOLTO)


In [ ]:
### 1) da fare : dopo aver sistemato questi casi
### riprendere il notebook     tirocinio/df omap/PerControllare.ipynb
### per controllare di quanti elementi abbiamo il relativo gold standard


In [ ]:
### ora consideriamo le annotazioni

In [9]:
# controllo coerenza per EsempioAnnotazione
EsempioAnnotazione.columns

Index(['TipoDiAnnotazione', 'ontologies', 'SOURCE', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui', 'type_ids', 'types', 'source_value',
       'detected_name', 'acc', 'context_similarity'],
      dtype='object')

In [10]:
# in EsempioAnnotazione 'SOURCE', 'AttributoNomeCompleto' deve essere una foreign key riferita a SCHEMI
len(EsempioAnnotazione), len(pd.merge(EsempioAnnotazione,SCHEMI))

(5031, 5031)

In [ ]:
# ok

In [22]:
### METTERE TUTTE LE ANNOTAZIONI FATTE in un unico file
### e ripetere il controllo precedente
### ora ovviamente stiamo considerando tutti gli schemi non solo OMOP








In [9]:
#### Leggere il file delle annotazioni rispetto a BioPOrtal che avevo fatto io
AnnotazioniConBioportal=pd.read_csv("http://dbgroup.ing.unimore.it/OMAP/AnnotazioniConBioportal.csv")
# controllo coerenza con schemi
len(AnnotazioniConBioportal), len(pd.merge(AnnotazioniConBioportal,SCHEMI))

(12694, 12694)

In [10]:
# avevo considerato solo OMOP
AnnotazioniConBioportal.groupby('SOURCE').agg(
    num_distinct_AttributoNomeCompleto=('AttributoNomeCompleto', 'nunique'),
    num_distinct_TipoDiAnnotazione=('TipoDiAnnotazione', 'nunique'),
    num_distinct_Ontologia=('Ontologia', 'nunique'),
    num_distinct_Codice=('Codice', 'nunique'),
    mean_codice_per_attributo=('Codice', lambda x: x.nunique() / x.groupby(AnnotazioniConBioportal['AttributoNomeCompleto']).ngroup().nunique())
)


,num_distinct_AttributoNomeCompleto,num_distinct_TipoDiAnnotazione,num_distinct_Ontologia,num_distinct_Codice,mean_codice_per_attributo
SOURCE,,,,,
OMOP,267,4,2,604,2.262172


In [11]:
AnnotazioniConBioportal.columns

Index(['TipoDiAnnotazione', 'Ontologia', 'SOURCE', 'AttributoNomeCompleto',
       'NumProg', 'Elemento', 'Codice', 'Description'],
      dtype='object')

In [12]:
annotazioniMedcat = pd.read_csv('files/annotazioni.csv', delimiter=",")
annotazioniMedcat.columns

Index(['TipoDiAnnotazione', 'ontologies', 'SOURCE', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui'],
      dtype='object')

In [13]:
## per mettere assieme tutto e confrontarle occorre uniformare i nomi di questi attributi
## con quelle delle tue annotazioni di medcat

EsempioAnnotazione.columns





Index(['TipoDiAnnotazione', 'ontologies', 'SOURCE', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui', 'type_ids', 'types', 'source_value',
       'detected_name', 'acc', 'context_similarity'],
      dtype='object')

In [31]:
## controlla e dicidi tu quali nomi tenere
## a me sembra che
## entity_id --> è il numero progressivo  NumProg per 'TipoDiAnnotazione', 'Ontologia', 'SOURCE', 'AttributoNomeCompleto',

## cui --> Codice  sono i codici dei concetti

## pretty_name --> Description  sono i nomi dei concetti

## 'source_value' --> Elemento   ??

## altri elementi delle annotazioni di medcat ('acc', 'context_similarity', ...) non hanno corrispodenza in Bioportal e quindi per queste assumeranno valore nullo 

In [14]:
EsempioAnnotazione

,TipoDiAnnotazione,ontologies,SOURCE,AttributoNomeCompleto,entity_id,pretty_name,cui,type_ids,types,source_value,detected_name,acc,context_similarity
0,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_birth_dt,1,Date of birth,184099003,['2680757'],['observable entity'],date of birth,date~of~birth,1.00,1.00
1,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_death_dt,1,Date of death,399753006,['2680757'],['observable entity'],date of death,date~of~death,1.00,1.00
2,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_esrd_ind,2,End-stage renal disease,46177005,['9090192'],['disorder'],end stage renal disease,end~stage~renal~disease,1.00,1.00
3,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-sp_state_code,0,State,398070004,['75168589'],['environment'],state,state,1.00,1.00
4,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_hi_cvrage_tot_mons,0,Total,255619001,['7882689'],['qualifier value'],total,total,0.99,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5026,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Synthea,providers-state,5,Abbreviation,398223008,['7882689'],['qualifier value'],abbreviation,abbreviation,1.00,1.00
5027,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Synthea,providers-zip,0,Patient,116154003,['31601201'],['person'],patient,patient,1.00,1.00
5028,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Synthea,providers-zip,2,Street address,398099009,['2680757'],['observable entity'],street address,street~address,1.00,1.00
5029,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Synthea,providers-utilization,0,Patient,116154003,['31601201'],['person'],patient,patient,1.00,1.00


# Unione dataframe medcat e bioportal



In [17]:
AnnotazioniConBioportal.rename(columns={"NumProg": "entity_id"}, inplace=True)


In [18]:
AnnotazioniConBioportal.rename(columns={"Ontologia": "ontologies"}, inplace=True)

In [19]:
AnnotazioniConBioportal.rename(columns={"Codice": "cui"}, inplace=True)

In [20]:
AnnotazioniConBioportal.rename(columns={"Description": "pretty_name"}, inplace=True)

In [21]:
AnnotazioniConBioportal.drop(columns=["Elemento"], inplace=True)

In [22]:
nuovo_ordine = ['TipoDiAnnotazione', 'ontologies', 'SOURCE', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui']

In [23]:
AnnotazioniConBioportal = AnnotazioniConBioportal[nuovo_ordine]


In [24]:
AnnotazioniConBioportal

,TipoDiAnnotazione,ontologies,SOURCE,AttributoNomeCompleto,entity_id,pretty_name,cui
0,BioportalAttributoNomeCompleto,LOINC,OMOP,person-person_id,1,Person,LP345156-6
1,BioportalAttributoNomeCompleto,LOINC,OMOP,person-person_id,2,Person,LP133774-2
2,BioportalAttributoNomeCompleto,LOINC,OMOP,person-person_id,3,Person,MTHU066373
3,BioportalAttributoNomeCompleto,LOINC,OMOP,person-person_id,4,Person:ID:Pt:^Patient:Nom,106193-6
4,BioportalAttributoNomeCompleto,LOINC,OMOP,person-person_id,5,Person,MTHU041262
...,...,...,...,...,...,...,...
12689,BioportalAttributo,SNOMEDCT,OMOP,provider-gender_source_value,4,Source (property),734842000
12690,BioportalAttributo,SNOMEDCT,OMOP,provider-gender_source_value,5,Source,260753009
12691,BioportalAttributo,SNOMEDCT,OMOP,provider-gender_source_concept_id,3,Gender,263495000
12692,BioportalAttributo,SNOMEDCT,OMOP,provider-gender_source_concept_id,4,Source (property),734842000


In [25]:
annotazioniMedcat

,TipoDiAnnotazione,ontologies,SOURCE,AttributoNomeCompleto,entity_id,pretty_name,cui
0,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_birth_dt,1,Date of birth,184099003
1,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_death_dt,1,Date of death,399753006
2,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_esrd_ind,2,End-stage renal disease,46177005
3,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-sp_state_code,0,State,398070004
4,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_hi_cvrage_tot_mons,0,Total,255619001
...,...,...,...,...,...,...,...
25403,Medcat_UMLS_Df_Full_ANC,UMLS,Synthea,providers-utilization,3,patient care,C0017313
25404,Medcat_UMLS_Df_Full_ANC,UMLS,Synthea,providers-utilization,5,Numbers,C0237753
25405,Medcat_UMLS_Df_Full_ANC,UMLS,Synthea,providers-utilization,7,Encounter,C1947978
25406,Medcat_UMLS_Df_Full_ANC,UMLS,Synthea,providers-utilization,9,Performed By,C1550369


In [34]:
annotazioniMedcat['annotator'] = 'MedCAT'
AnnotazioniConBioportal['annotator'] = 'Bioportal'

In [35]:
Annotazioni_full= pd.concat([annotazioniMedcat, AnnotazioniConBioportal], ignore_index=True)

In [36]:
Annotazioni_full.to_csv('files/annotazioni_full.csv', index=False)

In [37]:
Annotazioni_full


,TipoDiAnnotazione,ontologies,SOURCE,AttributoNomeCompleto,entity_id,pretty_name,cui,annotator
0,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_birth_dt,1,Date of birth,184099003,MedCAT
1,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_death_dt,1,Date of death,399753006,MedCAT
2,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_esrd_ind,2,End-stage renal disease,46177005,MedCAT
3,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-sp_state_code,0,State,398070004,MedCAT
4,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_hi_cvrage_tot_mons,0,Total,255619001,MedCAT
...,...,...,...,...,...,...,...,...
38097,BioportalAttributo,SNOMEDCT,OMOP,provider-gender_source_value,4,Source (property),734842000,Bioportal
38098,BioportalAttributo,SNOMEDCT,OMOP,provider-gender_source_value,5,Source,260753009,Bioportal
38099,BioportalAttributo,SNOMEDCT,OMOP,provider-gender_source_concept_id,3,Gender,263495000,Bioportal
38100,BioportalAttributo,SNOMEDCT,OMOP,provider-gender_source_concept_id,4,Source (property),734842000,Bioportal


appunto= ho notato che in bioportal le ontologie sono in snomed e loinc

# 21 Marzo

Aggiunta del campo annotator

Creiamo un nuovo campo chiamato annotator, che viene estratto direttamente da TipoDiAnnotazione, poiché quest’ultimo determina univocamente l’annotatore utilizzato.

Analisi della relazione tra source_value e detected_name

Verifichiamo se source_value implica sempre detected_name o viceversa. Questo ci aiuterà a capire se un source_value corrisponde sempre a un detected_name o se esistono eccezioni.


In [30]:
Annotazioni=EsempioAnnotazione

# Aggiunta del campo 'annotator' estratto da 'TipoDiAnnotazione'
Annotazioni['annotator'] = 'MedCAT'

# Verifica se ogni 'source_value' ha sempre lo stesso 'detected_name'
source_determines_detected = (
    Annotazioni.groupby('source_value')['detected_name']
    .nunique()
    .max() == 1
)

# Verifica se ogni 'detected_name' ha sempre lo stesso 'source_value'
detected_determines_source = (
    Annotazioni.groupby('detected_name')['source_value']
    .nunique()
    .max() == 1
)

# Stampa i risultati
print(f"Source Value determina univocamente Detected Name? {source_determines_detected}")
print(f"Detected Name determina univocamente Source Value? {detected_determines_source}")

Source Value determina univocamente Detected Name? True
Detected Name determina univocamente Source Value? False


Dall’analisi precedente, abbiamo osservato che detected_name non determina univocamente source_value, il che significa che lo stesso detected_name può essere associato a più valori di source_value.

Per approfondire questa relazione, individuiamo quale detected_name è associato al maggior numero di valori distinti di source_value. Questo ci aiuterà a capire quali entità rilevate (detected_name) sono più ambigue o hanno una maggiore variabilità nei corrispondenti valori di origine (source_value).

In [31]:
# Raggruppa per 'detected_name' e raccoglie i valori distinti di 'source_value'
detected_source_values = (
    Annotazioni.groupby('detected_name')['source_value']
    .agg(lambda x: sorted(x.unique()))  # Ordina i valori distinti
    .reset_index()
)

# Conta il numero di valori distinti per ogni 'detected_name'
detected_source_values['num_distinct_source_values'] = detected_source_values['source_value'].apply(len)

# Seleziona i tre 'detected_name' con il maggior numero di 'source_value' distinti
top_detected_names = detected_source_values.nlargest(3, 'num_distinct_source_values')

# Mostra il risultato
top_detected_names[['detected_name', 'num_distinct_source_values', 'source_value']]

,detected_name,num_distinct_source_values,source_value
80,diagnosis,3,"[diafnosis, diagnoses, diagnosis]"
201,order,3,"[order, orderid, orders]"
4,address,2,"[address, addresses]"


Dall'analisi precedente, emerge chiaramente che il *Detected Name* non è altro che una rielaborazione sintattica del *Source Value*.
Le trasformazioni osservate includono:
- La conversione di termini dal singolare al plurale (o viceversa).
- La rimozione di identificatori o codici finali.
- La normalizzazione di alcune variazioni linguistiche o abbreviazioni.

Questa osservazione suggerisce che il *Detected Name* non introduce nuova informazione semantica, ma rappresenta semplicemente una versione riformattata del *Source Value*.

### Analisi della relazione tra *Detected Name* e *CUI*

Dopo aver analizzato la relazione tra *Detected Name* e *Source Value*, passiamo ora a un'analisi ancora più cruciale: il rapporto tra *Detected Name* e *CUI* (Concept Unique Identifier).

Questo studio è fondamentale per comprendere se la notazione di un determinato *Detected Name* varia in base al contesto in cui è utilizzato, ovvero se cambia a seconda dell’attributo associato, del suo elemento, del nome o della descrizione.

L'obiettivo è verificare se esiste una relazione di determinazione tra *Detected Name* e *CUI*:
- Un *Detected Name* corrisponde sempre a un unico *CUI*?
- Oppure uno stesso *Detected Name* può essere associato a *CUI* differenti in base al contesto?
- Viceversa, un *CUI* è sempre legato a un unico *Detected Name*?

Per rispondere a queste domande, analizziamo la relazione tra i due attributi.

In [32]:
Annotazioni = EsempioAnnotazione

# Verifica se ogni 'detected_name' ha sempre lo stesso 'cui'
detected_determines_cui = (
    Annotazioni.groupby('detected_name')['cui']
    .nunique()
    .max() == 1
)

# Verifica se ogni 'cui' ha sempre lo stesso 'detected_name'
cui_determines_detected = (
    Annotazioni.groupby('cui')['detected_name']
    .nunique()
    .max() == 1
)

# Stampa i risultati
print(f"Detected Name determina univocamente CUI? {detected_determines_cui}")
print(f"CUI determina univocamente Detected Name? {cui_determines_detected}")

Detected Name determina univocamente CUI? False
CUI determina univocamente Detected Name? False


In [33]:
# Trova i detected_name associati a più di un cui
detected_with_multiple_cui = (
    Annotazioni.groupby('detected_name')['cui']
    .nunique()
    .reset_index()
    .query("cui > 1")
    .head(2)  # Seleziona almeno un paio di esempi
)

# Mostra per ciascuno l'elenco distinto di cui e pretty_name
for detected in detected_with_multiple_cui['detected_name']:
    cui_pretty_name_list = Annotazioni.loc[Annotazioni['detected_name'] == detected, ['types','cui', 'pretty_name']].drop_duplicates()
    print(f"Detected Name: {detected} -> CUI and Pretty Name: {cui_pretty_name_list.values.tolist()}")

Detected Name: prescription -> CUI and Pretty Name: [["['attribute']", 260885003, 'Prescription'], ["['procedure']", 16076005, 'Prescription']]


### Analisi della Relazione tra "Detected Name" e "CUI"

Abbiamo trovato un risultato interessante che ci mostra come un **Detected Name** (ad esempio "Prescription") può essere associato a **CUI** diversi, ma con **tipi** differenti. Il risultato è il seguente:


Detected Name: prescription -> CUI and Pretty Name: [["['attribute']", 260885003, 'Prescription'], ["['procedure']", 16076005, 'Prescription']]


Questo ci dice che il termine "Prescription" è legato a due **CUI** distinti:

1. **CUI 260885003** con tipo **'attribute'**;
2. **CUI 16076005** con tipo **'procedure'**.

Questo comportamento è molto interessante perché significa che lo stesso **Detected Name**, in questo caso "Prescription", può essere interpretato in modi diversi a seconda del **tipo** associato. In pratica, "Prescription" potrebbe essere:

- Un **attributo**, se rappresenta una caratteristica o una proprietà di un farmaco;
- Una **procedura**, se indica l'atto medico di prescrivere un farmaco.


Per comprendere appieno questo comportamento, è fondamentale capire cosa significa "tipo" in **UMLS** (Unified Medical Language System). In UMLS, ogni concetto viene classificato sotto un determinato **tipo semantico** (ad esempio, "attributo" o "procedura"). Questi tipi sono utili per distinguere tra concetti che condividono lo stesso nome ma hanno significati diversi a seconda del contesto.

In questo caso, "Prescription" viene associato sia a un **attributo** che a una **procedura**. Quindi, lo stesso termine può avere significati diversi in base al tipo semantico a cui è collegato.


1. **Studiare meglio UMLS**: È necessario approfondire la documentazione di UMLS per capire meglio come vengono assegnati i tipi semantici ai **CUI** e come queste classificazioni influenzano l'interpretazione dei concetti.

2. **Esaminare la letteratura**: Sarà utile consultare articoli e risorse che parlano di come UMLS gestisce i **tipi semantici**, per avere una visione più chiara del sistema.

3. **Chiarire il contesto**: Una volta compreso meglio il concetto di tipo in UMLS, potremo spiegare come un **Detected Name** come "Prescription" può essere associato a **CUI** diversi, ognuno con un significato distinto a seconda del tipo.

In [48]:
# Trova i cui associati a più di un detected_name
cui_with_multiple_detected = (
    Annotazioni.groupby('cui')['detected_name']
    .nunique()
    .reset_index()
    .query("detected_name > 1")
    .head(2)  # Seleziona almeno un paio di esempi
)

# Mostra per ciascuno l'elenco distinto di detected_name
for cui_value in cui_with_multiple_detected['cui']:
    detected_list = Annotazioni.loc[Annotazioni['cui'] == cui_value, 'detected_name'].unique()
    print(f"CUI: {cui_value} -> Detected Names: {list(detected_list)}")

CUI: 3457005 -> Detected Names: ['refer', 'referring']
CUI: 7882003 -> Detected Names: ['identify', 'identified']


In [42]:
df_filtrato = Annotazioni.loc[Annotazioni['cui'] == 7882003]
df_filtrato

,TipoDiAnnotazione,ontologies,SOURCE,AttributoNomeCompleto,entity_id,pretty_name,cui,type_ids,types,source_value,detected_name,acc,context_similarity,annotator
342,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Mimic,callout-submit_wardid,4,Identified,7882003,['7882689'],['qualifier value'],identifies,identify,1.0,1.0,MedCAT
350,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Mimic,callout-curr_wardid,4,Identified,7882003,['7882689'],['qualifier value'],identifies,identify,1.0,1.0,MedCAT
357,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Mimic,callout-callout_wardid,4,Identified,7882003,['7882689'],['qualifier value'],identifies,identify,1.0,1.0,MedCAT
458,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Mimic,chartevents-value,5,Identified,7882003,['7882689'],['qualifier value'],identified,identified,1.0,1.0,MedCAT
541,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Mimic,d_cpt-category,6,Identified,7882003,['7882689'],['qualifier value'],identifies,identify,1.0,1.0,MedCAT
625,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Mimic,d_items-conceptid,2,Identified,7882003,['7882689'],['qualifier value'],identified,identified,1.0,1.0,MedCAT
668,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Mimic,datetimeevents-value,6,Identified,7882003,['7882689'],['qualifier value'],identified,identified,1.0,1.0,MedCAT
887,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Mimic,labevents-value,10,Identified,7882003,['7882689'],['qualifier value'],identified,identified,1.0,1.0,MedCAT
1051,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Mimic,outputevents-value,3,Identified,7882003,['7882689'],['qualifier value'],identified,identified,1.0,1.0,MedCAT
1056,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Mimic,outputevents-iserror,2,Identified,7882003,['7882689'],['qualifier value'],identified,identified,1.0,1.0,MedCAT


### Da fare

continuare con analisi e ragionamenti di questo tipo. questo ci consentira di descrivere l'annotator (in questo caso MedCat) sulla base dei risultati ottenuti applicandolo ad un caso reale

Il passo successivo è quello di fare velocemente analisi simili anche per l'altro Annotator Bioportal e fare un minimo di confronto:
queste analisi dovrebbero essere veloci perchè se mettiamo tutto in uno stesso formato, basta far rgirare il codice per il sottoinsieme relativo a Bioportal


**Analisi e Confronto tra Annotatori: MedCat e Bioportal**

Il nostro obiettivo è esaminare e confrontare i risultati ottenuti dai due annotatori, MedCat e Bioportal, al fine di capire meglio le differenze e le similitudini tra le due annotazioni e come queste possano influenzare l’interpretazione dei dati.

Passaggi da seguire
	1.	Analisi dei dati per MedCat (già completata):
Abbiamo già effettuato delle analisi approfondite sui dati annotati da MedCat, esaminando la relazione tra Detected Name e CUI, nonché l’associazione tra i vari CUI e i Pretty Name. Questi risultati ci hanno permesso di comprendere come il sistema MedCat etichetta e gestisce le informazioni relative ai concetti medici, evidenziando le differenze tra i tipi di concetti come “attributo” e “procedura” per lo stesso Detected Name.
	2.	Ripetizione delle analisi per Bioportal:
Il passo successivo è applicare le stesse analisi ai dati annotati da Bioportal. Per fare questo, dovremo filtrare i dati per l’annotatore Bioportal e ripetere le stesse operazioni effettuate per MedCat. Questo ci permetterà di confrontare i risultati ottenuti dai due annotatori e osservare se ci sono differenze significative nella classificazione dei concetti medici e nelle relazioni tra Detected Name, CUI e Pretty Name. Il codice per effettuare questa analisi è simile a quello utilizzato per MedCat:

Prima di procedere con il confronto, è importante verificare se MedCat e Bioportal utilizzano le stesse versioni dell’ontologia SNOMED.  Le versioni di SNOMED utilizzate da entrambi gli annotatori devono essere confrontate attraverso la documentazione ufficiale o le informazioni fornite dai rispettivi sistemi. Se le versioni sono diverse, sarà necessario tenerne conto quando si interpretano i risultati del confronto. E quando poi si confrontano con il gold standard
